# Keras 개요

- [keras.io](http://keras.io/)
- [github.com/fchollet/keras](https://github.com/fchollet/keras)

- keras는 Python Theano 기반의 deep learning 도구이다
- 쉽고 빠르게 Prototype을 만들수 있다 
- convolution network과 recurrent networks 그리고 합쳐진 형태도 지원한다 
- 임의의 연결이 가능하다 (supports arbitrary connectivity schemes)
- CPU와 GPU연산을 지원한다

### 설치하기

- 기본적으로 numpy, scipy, theano 가 필요하며
- 선택적으로 HDF5, h5py, cuDNN, 등이 필요하다
- 설치는 
- sudo pip install keras
- 하면 알아서 다 깔아 준다 
- 또는 
- git clone  https://github.com/fchollet/keras.git
- cd keras  (download 받은 폴더로 간다)
- sudo python setup.py install


## 사용하기

In [1]:
from keras.models import Sequential

model = Sequential()


In [2]:
from keras.layers.core import Dense, Activation

model.add(Dense(output_dim=64, input_dim=100, init="glorot_uniform"))
model.add(Activation("relu"))
model.add(Dense(output_dim=10, init="glorot_uniform"))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='sgd')

/home/zena/Python2.7/lib/python2.7/site-packages/keras/layers/core.pyc in get_output(self, train)
    409     def get_output(self, train=False):
    410         X = self.get_input(train)
--> 411         return self.activation(X)
    412 
    413     def get_config(self):

/home/zena/Python2.7/lib/python2.7/site-packages/keras/activations.pyc in relu(x)
     18 
     19 def relu(x):
---> 20     return T.nnet.relu(x)
     21 
     22 

AttributeError: 'module' object has no attribute 'relu'

$ sudo pip uninstall theano
$ sudo pip install git+git://github.com/Theano/Theano.git

Kernel reset

In [3]:
model.compile(loss='categorical_crossentropy', optimizer='sgd')

In [4]:
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))


model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)

model.train_on_batch(X_batch, Y_batch)

objective_score = model.evaluate(X_test, Y_test, batch_size=32)

classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)

[imdb_cnn.py](https://github.com/fchollet/keras/blob/cb77f7d7e2933078838e836018fdfa7075e5385f/examples/imdb_cnn.py)

In [9]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.datasets import imdb

'''
    This example demonstrates the use of Convolution1D
    for text classification.
    Run on GPU: THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python imdb_cnn.py
    Get to 0.8330 test accuracy after 3 epochs. 100s/epoch on K520 GPU.
'''

# set parameters:
max_features = 5000
maxlen = 100
batch_size = 32
embedding_dims = 100
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 3

print("Loading data...")
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features,
                                                      test_split=0.2)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.25))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode="valid",
                        activation="relu",
                        subsample_length=1))
# we use standard max pooling (halving the output of the previous layer):
model.add(MaxPooling1D(pool_length=2))

# We flatten the output of the conv layer, so that we can add a vanilla dense layer:
model.add(Flatten())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', class_mode="binary")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, validation_data=(X_test, y_test))


Loading data...
20000 train sequences
5000 test sequences
Pad sequences (samples x time)
X_train shape: (20000, 100)
X_test shape: (5000, 100)
Build model...
Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 824s - loss: 0.6401 - acc: 0.5930 - val_loss: 0.5144 - val_acc: 0.7442
Epoch 2/3
20000/20000 [==============================] - 823s - loss: 0.3908 - acc: 0.8255 - val_loss: 0.3615 - val_acc: 0.8344
Epoch 3/3
20000/20000 [==============================] - 817s - loss: 0.3173 - acc: 0.8636 - val_loss: 0.3788 - val_acc: 0.8256
